In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

In [2]:
driver = webdriver.Chrome('./chromedriver.exe')

In [3]:
url = 'https://www.tripadvisor.com/Search?q=south%20korea&searchSessionId=470ABCDA6E9199ADAD85A571C57369281618334553723ssid&searchNearby=false&sid=EC37354CF84B431D9D6FC80F5003425E1618334567611&blockRedirect=true&ssrc=ac&rf=1'

In [4]:
def get_reviews(driver):
    results = driver.page_source
    soup = BeautifulSoup(results, 'lxml')
    a = soup.find_all('a')

    url_list = []
    for j in range(0,len(a)):
        try:
            onclick = a[j].attrs['href']
            start = onclick.find('AttractionProductReview')
            end = onclick.find('.html', start)
            url = onclick[start:end]
            if url != '':
                url_list.append(url)
            else:
                continue
        except:
            pass

    result = []
    for i in range(0,30): 
        tour_title = soup.select('div.result-title')[i].text
        tour_location = soup.select('div.address-text')[i].text
        tour_url = url_list[i]
        
        result.append([tour_title, tour_location, tour_url])
        i += 1
    
    df = pd.DataFrame(result)
    df.columns = ['tour_title', 'tour_location', 'tour_url']
    
    return df

def next_reviews(driver):

    result_all = pd.DataFrame()
    
    for page in range(0,34):
        pg_review = get_reviews(driver)
        result_all = pd.concat([result_all, pg_review], axis = 0)
        print(f'{page} page done')
        
        try:
            driver.find_element_by_xpath("//a[@class='ui_button nav next primary ']").click()
            time.sleep(3)
            page += 1
            
        except:
            break
            
    return result_all

In [5]:
driver.get(url)
driver.implicitly_wait(10)
time.sleep(3)

reviews = next_reviews(driver)
reviews

0 page done
1 page done
2 page done
3 page done
4 page done
5 page done
6 page done
7 page done
8 page done
9 page done
10 page done
11 page done
12 page done
13 page done
14 page done
15 page done
16 page done
17 page done
18 page done
19 page done
20 page done
21 page done
22 page done
23 page done
24 page done
25 page done
26 page done
27 page done
28 page done
29 page done
30 page done
31 page done
32 page done
33 page done


,tour_title,tour_location,tour_url
0,Seoul Private Flexible Adventure Tour,"Seoul, South Korea",AttractionProductReview-g294197-d12521956-Seou...
1,Korean History & Heritage Tour,"Seoul, South Korea",AttractionProductReview-g294197-d15883739-Kore...
2,Fully Customizable Private Tour From Seoul to ...,"Seoul, South Korea",AttractionProductReview-g294197-d12649287-Full...
3,Half-Day Korean DMZ Tour from Seoul,"Seoul, South Korea",AttractionProductReview-g294197-d11989360-Half...
4,Korean Demilitarized Zone (DMZ) Half-Day Tour ...,"Seoul, South Korea",AttractionProductReview-g294197-d11454650-Kore...
...,...,...,...
25,"Cotopaxi full day tour from Quito, small groups","Quito, Pichincha Province, Ecuador",AttractionProductReview-g294308-d11764163-Coto...
26,O'Reilly's & Lamington National Park from Bris...,"Brisbane, Brisbane Region, Queensland, Australia",AttractionProductReview-g255068-d11446501-O_Re...
27,Hartley's Crocodile Adventure Half-Day Tour,"Cairns, Cairns Region, Queensland, Australia",AttractionProductReview-g255069-d11447728-Hart...
28,SuperSaver: Small Group Silfra Snorkeling and ...,"Hveragerdi, South Region, Iceland",AttractionProductReview-g315849-d11452168-Supe...


In [9]:
reviews.to_excel('tour_titles_all_url.xlsx')

In [10]:
reviews_korea = reviews[reviews['tour_location'].str.contains('South Korea')]
reviews_korea

,tour_title,tour_location,tour_url
0,Seoul Private Flexible Adventure Tour,"Seoul, South Korea",AttractionProductReview-g294197-d12521956-Seou...
1,Korean History & Heritage Tour,"Seoul, South Korea",AttractionProductReview-g294197-d15883739-Kore...
2,Fully Customizable Private Tour From Seoul to ...,"Seoul, South Korea",AttractionProductReview-g294197-d12649287-Full...
3,Half-Day Korean DMZ Tour from Seoul,"Seoul, South Korea",AttractionProductReview-g294197-d11989360-Half...
4,Korean Demilitarized Zone (DMZ) Half-Day Tour ...,"Seoul, South Korea",AttractionProductReview-g294197-d11454650-Kore...
...,...,...,...
26,Suwon Hwaseong Fortress and Korean Folk Villag...,"Seoul, South Korea",AttractionProductReview-g294197-d11461630-Suwo...
27,Korean Palace and Temple Tour in Seoul: Gyeong...,"Seoul, South Korea",AttractionProductReview-g294197-d12463701-Kore...
1,Korean Folk Village,"Seoul, South Korea",AttractionProductReview-g294197-d13159855-Kore...
6,History and Culture Tour,"Seoul, South Korea",AttractionProductReview-g294197-d20232226-Hist...


In [11]:
reviews_korea.to_excel('tour_titles_url_korea.xlsx')